## 1. gensim 키워드 추출 위한 전처리

In [ ]:
#리뷰 데이터 불러오기
import pandas as pd
df = pd.read_csv("C:/Users/user/Documents/team/reviews/gobchang_review.csv",encoding= "utf-8")

# 특수문자 제거
df['comment']=df['comment'].str.replace("[^ ㄱ-ㅣ가-힣A-Za-z]", " ")

# 코멘트 리스트화
df_list = []
for i in range(len(df['comment'])):
    df_list.append(df['comment'][i])

In [ ]:
#토큰화하여 저장
from konlpy.tag import Okt

okt_list= []
okt = Okt()

for i in range(len(df_list)):
    okt_list.append(okt.morphs(df_list[i]))

In [ ]:
#토큰화 후 품사태깅
pos_list=[]
for i in okt_list:
    oktTag = []
    for token in i:
        oktTag+= okt.pos(token)
    pos_list.append(oktTag)

In [ ]:
#포함될 품사 선정 및 불용어 선정
stopPos = ['Noun','Verb','Adjective']
stopWord = ['이','거','기고','멓으','요요','입니다','같아요','했어요']

gop=[]
for a in pos_list:
    곱창=[]
    for tag in a:
        if tag[1] in stopPos:
            if tag[0] not in stopWord:        
                if len(tag[0])>1:
                    곱창.append(tag[0])  
    gop.append(곱창)

## 2. 키워드 추출 및 5개 키워드 선정

In [ ]:
#하나의 텍스트로 만든 후 키워드 추출
word = str(gop).replace(',',' ').replace('[','').replace(']','').replace("'",'')

from gensim.summarization import keywords
keyword = keywords(word,words = 100).split()
print(keyword)

In [ ]:
#gensim 을 통해 추출된 키워드 중 5개 키워드 선정
a = df[df['comment'].str.contains('다이어트')]
a = a.append(df[df['comment'].str.contains('샐러드')])
a = a.append(df[df['comment'].str.contains('브랜드')])
a = a.append(df[df['comment'].str.contains('강아지')])
a = a.append(df[df['comment'].str.contains('단백질')])

In [ ]:
#키워드가 포함된 리뷰를 새로운 데이터프레임으로 만듦
a.drop_duplicates(inplace = True)
a= a['comment']
a.reset_index(drop = True,inplace=True)

In [ ]:
#코멘트 리스트화
a_list = []
for i in range(len(a)):
    a_list.append(a[i])

In [ ]:
#토큰화하여 저장
from konlpy.tag import Okt

okt_list= []
okt = Okt()

for i in range(len(a_list)):
    okt_list.append(okt.morphs(a_list[i]))
print(okt_list)

## 3.불용어 처리 및 저장

In [ ]:
#토큰화후 품사태깅
pos_list=[]
for i in okt_list:
    oktTag = []
    for token in i:
        oktTag+= okt.pos(token)
    pos_list.append(oktTag)

In [ ]:
#단어의 빈도 계산하여 저장하고 빈도수대로 정렬한 리스트 생성
word_list = word.split()
words_count={}
for word in word_list:
    if word in words_count:
        words_count[word] += 1
    else:
        words_count[word] = 1
sorted_words = sorted([(k,v) for k,v in words_count.items()], key=lambda word_count: -word_count[1])

In [ ]:
#직접 선정한 불용어 및 일정 빈도수 이하로 나온 경우 stopwod에 추가
stopWord = ['진짜','이나','뭔가','같아요','지도','인지','기지','주신','나은','주신','경우','우선','정말','뭔가','노력','인지','기지','오늘','구울','베스트','배달','지도','구울', '예정','나은','주신','경우','여기','자체','벌써','여기','기고','인분','우리', '저희','일단',
            '원래','샤를','안나','쓰기','했습','겠습','일해','발도','이예','자꾸','나용','매우','거리','주사','레미','비도','라서','살맛','요요','비도','살이','라미','어요','거리','온도','대로','다만','사진','운빵','안쪽','이나','몇개','보고','정도','다른','하기','해봤어요','하네요','잇어요','들어요','바로',
            '에요','여러','정도','려고','덩어리','사서','살로','덩이','사먹','하는','정말','먹을','했는데','두고','하나','인분','생각','먹기', '아주','정말','만해','사람','먹음','한번','있어','같네요','같습니다','같아요', '그냥','조금','좀','해서','하니','합니다','입니다','에요','이건','해도','이거','방금','이제','처럼']

for i in range(len(sorted_words)) :
    if sorted_words[i][1] <= 15 :
        stopWord.append(sorted_words[i][0])

In [ ]:
##포함될 품사 선정 및 불용어 처리
stopPos = ['Noun']
gop=[]
for a in pos_list:
    곱창=[]
    for tag in a:
        if tag[1] in stopPos:
            if tag[0] not in stopWord:        
                if len(tag[0])>1:
                    곱창.append(tag[0])  
    gop.append(곱창)

In [ ]:
#csv 파일로 저장
data= pd.DataFrame({"labels":gop})
data.to_csv("곱창_키워드5_토큰화_SSG.csv", index=False, encoding="utf-8-sig")